In [13]:
import requests

from bs4 import BeautifulSoup

import pandas as pd
import folium

### Finding and extracting contents of table in Wiki page


In [2]:
URL = r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')


In [3]:
data=pd.read_html(URL, attrs={"class": "wikitable"})
data=data[0]     

In [4]:
data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Checking where *Not assigned* appear in *Borough*

In [5]:
data[data['Borough'] =="Not assigned"]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


#### Removing rows in which the *Borough* field is "Not assigned"

In [6]:
data.drop(data[data['Borough'] =="Not assigned"].index,axis=0,inplace=True)
data.reset_index(drop=True,inplace=True)

In [7]:
data.shape

(103, 3)

#### Geocoder was not being able to retrieve the latitude,longitude so we'll use the CSV

In [9]:
import geocoder # import geocoder


# initialize your variable to None


# loop until you get the coordinates

#for postal_code in postal_code:
#    try:
#        for i in range(2):
 #           g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
 #           lat_lng_coords = g.latlng
#    except:
 #       print("Error")
        

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]


In [10]:
coord=pd.read_csv(r'https://cocl.us/Geospatial_data')
    

#### After reading the .csv file, we'll merge the coord and data dfs into one

In [11]:
data=pd.merge(data,coord,
        how='inner',
        on='Postal Code')

In [12]:
data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [15]:
locations = data[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)

103

In [27]:
map = folium.Map(location=locationlist[0], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.CircleMarker(locationlist[point], popup=data['Neighbourhood'][point]).add_to(map)
map